<a href="https://colab.research.google.com/github/itimes-digital/mineracao-texto/blob/main/analise_de_texto_para_classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [190]:
import nltk
import pandas as pd
import re
from unicodedata import normalize

In [191]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [192]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

### Preparação dos dados

In [193]:
base = pd.read_csv('sample_data/lista_classificada.csv', sep=';', engine='python')
baseCompleta = pd.read_csv('sample_data/DesafioAnderson2.csv', sep=';', engine='python', encoding='utf8')

In [194]:
base.head()

,Categoria,Comentario
0,"Cadastro, Login, Senha",Nao recebi email para validar o cadastro NÃO C...
1,"Cadastro, Login, Senha",Não consigo concluir o cadastro.
2,"Loja C&R, Estoque",Nunca realizei compras pelo app. Pois a forma ...
3,Entrega,"Muito demora pra entregar.Farmácia é urgência,..."
4,Entrega,"Péssimo no que se refere ao item ENTREGA, pois..."


In [195]:
base['Comentario'] = base['Comentario'].map(lambda x: re.sub('[,\.!?-]', '', x))
base['Comentario'] = base['Comentario'].map(lambda x: normalize('NFKD', x.lower()).encode('ASCII','ignore').decode('ASCII'))

In [196]:
base.head()

,Categoria,Comentario
0,"Cadastro, Login, Senha",nao recebi email para validar o cadastro nao c...
1,"Cadastro, Login, Senha",nao consigo concluir o cadastro
2,"Loja C&R, Estoque",nunca realizei compras pelo app pois a forma d...
3,Entrega,muito demora pra entregarfarmacia e urgencia q...
4,Entrega,pessimo no que se refere ao item entrega pois ...


In [197]:
base['Categoria'].value_counts()

Lentidão, Trava, Crash, Performance    121
Entrega                                 85
Pedido cancelamento                     39
Erro finalização                        38
Erro PIX                                34
Loja C&R, Estoque                       27
Preço, Prazo, Promoção, Stix            21
Erro cartão de crédito                  11
Cesta                                   10
Busca                                    7
Novas Funcionalidades                    6
Cadastro, Login, Senha                   5
Problema não detalhado                   3
Name: Categoria, dtype: int64

In [198]:
base = base.sample(frac=1)
base_treinamento = base;

In [199]:
#base_treinamento = base.iloc[:350,:]
base_teste = base.iloc[350:,:]

In [200]:
base_treinamento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 407 entries, 192 to 321
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Categoria   407 non-null    object
 1   Comentario  407 non-null    object
dtypes: object(2)
memory usage: 9.5+ KB


In [201]:
base_teste.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 121 to 321
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Categoria   57 non-null     object
 1   Comentario  57 non-null     object
dtypes: object(2)
memory usage: 1.3+ KB


In [202]:
base_treinamento = base_treinamento.reset_index(drop=True)
base_teste = base_teste.reset_index(drop=True)

In [203]:
base_teste.head()

,Categoria,Comentario
0,Erro PIX,nao consigo copiar codigo pix para pagar o med...
1,Entrega,nao esta deixando eu optar entre entregar ou r...
2,"Lentidão, Trava, Crash, Performance",aplicativo lento
3,Cesta,cansativo se nao tem nao deixe colocar no carr...
4,"Lentidão, Trava, Crash, Performance",custei acessar o app


In [204]:
stopwords = nltk.corpus.stopwords.words('portuguese')
#print(stopwords)

In [205]:
def removeStopWordsAplicandoStemmer(base):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for i in range(len(base)):
        sem_stop = [str(stemmer.stem(p)) for p in base.loc[i,"Comentario"].split() if p not in stopwords]
        frases.append((sem_stop, base.loc[i,"Categoria"]))
    return frases;

In [206]:
frasesComStemmingTreinamento = removeStopWordsAplicandoStemmer(base_treinamento);
frasesComStemmingTeste = removeStopWordsAplicandoStemmer(base_teste);
#print(frasesComStemming)

In [207]:
def buscaPalavras(frases):
    todasPalavras = [];
    for (palavras, categoria) in frases:
        todasPalavras.extend(palavras);
    return todasPalavras;

In [208]:
palavrasTreinamento = buscaPalavras(frasesComStemmingTreinamento);
palavrasTeste = buscaPalavras(frasesComStemmingTeste);
#print(palavras)

In [209]:
def buscaFrequencia(palavras):
    palavras = nltk.FreqDist(palavras);
    return palavras;

In [210]:
frequenciaTreinamento = buscaFrequencia(palavrasTreinamento);
frequenciaTeste = buscaFrequencia(palavrasTeste);
#print(frequencia.most_common(50))

In [211]:
def buscaPalavrasUnicas(frequencia):
    freq = frequencia.keys();
    return freq;

In [212]:
palavrasUnicasTreinamento = buscaPalavrasUnicas(frequenciaTreinamento);
palavrasUnicasTeste = buscaPalavrasUnicas(frequenciaTeste);
#print(palavrasUnicas)

In [213]:
def extratorPalavras(documento):
    doc = set(documento);
    caracteristicas = {};
    for palavras in palavrasUnicasTreinamento:
        caracteristicas['%s' % palavras] = (palavras in doc);
    return caracteristicas;

In [214]:
caracteristicasFrases = extratorPalavras({'am', 'nov', 'dia'});
#print(caracteristicasFrases)

### Realizando a classificação

In [215]:
baseCompletaTreinamento = nltk.classify.apply_features(extratorPalavras, frasesComStemmingTreinamento);
baseCompletaTeste = nltk.classify.apply_features(extratorPalavras, frasesComStemmingTeste);
#print(baseCompleta);

In [216]:
classificador = nltk.NaiveBayesClassifier.train(baseCompletaTreinamento);

In [217]:
print(classificador.labels())

['Lentidão, Trava, Crash, Performance', 'Entrega', 'Loja C&R, Estoque', 'Cesta', 'Preço, Prazo, Promoção, Stix', 'Erro finalização', 'Pedido cancelamento', 'Erro cartão de crédito', 'Erro PIX', 'Busca', 'Novas Funcionalidades', 'Problema não detalhado', 'Cadastro, Login, Senha']


In [218]:
 #usando a mesma amostra do modelo treinado
print(nltk.classify.accuracy(classificador, baseCompletaTreinamento))

0.7002457002457002


In [219]:
#usando a amostra de teste
print(nltk.classify.accuracy(classificador, baseCompletaTeste)) 

0.631578947368421


In [220]:
print(classificador.show_most_informative_features(10))

Most Informative Features
                  cancel = True           Pedido : Lentid =     78.3 : 1.0
                   carta = True           Erro c : Entreg =     45.4 : 1.0
                     pix = True           Erro P : Lentid =     45.3 : 1.0
                   acess = True           Cadast : Lentid =     33.9 : 1.0
                   retir = True           Proble : Lentid =     30.5 : 1.0
                  adiant = True           Proble : Lentid =     30.5 : 1.0
                     peg = True           Proble : Lentid =     30.5 : 1.0
                  estoqu = True           Loja C : Lentid =     30.5 : 1.0
                  entreg = True           Entreg : Lentid =     30.2 : 1.0
                    salv = True           Novas  : Lentid =     29.0 : 1.0
None


In [ ]:
erros = []

for (frase, classe) in baseCompletaTeste:
    previsao = classificador.classify(frase)

    if previsao != classe:
        erros.append((classe, previsao, frase))

for (flasse, previsao, frase) in erros:
    # Classe real | previsão da classe | frase
    print(classe, previsao, frase)

In [222]:
from nltk.metrics import ConfusionMatrix

esperado = []
previsto = []

for (frase, classe) in baseCompletaTeste:
    previsao = classificador.classify(frase);
    previsto.append(previsao);
    esperado.append(classe);

matriz = ConfusionMatrix(esperado, previsto)
print(matriz)

                                    |                       L             |
                                    |                       e             |
                                    |                       n             |
                                    |                       t             |
                                    |                       i             |
                                    |                       d             |
                                    |                       ã             |
                                    |                       o           P |
                                    |                       ,           r |
                                    |                                   e |
                                    |                       T           ç |
                                    |                       r           o |
                                    |                       a           , |
            

### Montagem de uma frase para testar classificação.

In [223]:
testeFrase = 'Meu pedido não fecha adoentado'

In [224]:
def aplicandoStemmerFrase(frase):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for (palavrasTreinamento) in frase.split():
        com_stem_sem_stop = [p for p in palavrasTreinamento.split()];
        frases.append(str(stemmer.stem(com_stem_sem_stop[0])));
    return frases;

In [225]:
fraseComStemming = aplicandoStemmerFrase(testeFrase)
print(fraseComStemming)

['meu', 'ped', 'não', 'fech', 'adoent']


In [226]:
novaFrase = extratorPalavras(fraseComStemming);
#print(novaFrase)

In [227]:
print(classificador.classify(novaFrase))

Lentidão, Trava, Crash, Performance


In [228]:
distribuicao = classificador.prob_classify(novaFrase)
for classe in distribuicao.samples():
    print("%s: %f" % (classe, distribuicao.prob(classe)))

Lentidão, Trava, Crash, Performance: 0.999878
Entrega: 0.000122
Loja C&R, Estoque: 0.000000
Cesta: 0.000000
Preço, Prazo, Promoção, Stix: 0.000000
Erro finalização: 0.000000
Pedido cancelamento: 0.000000
Erro cartão de crédito: 0.000000
Erro PIX: 0.000000
Busca: 0.000000
Novas Funcionalidades: 0.000000
Problema não detalhado: 0.000000
Cadastro, Login, Senha: 0.000000


### Montagem dos dados novos para classificação

In [229]:
baseCompleta['Comentario'] = baseCompleta['Comentario'].map(lambda x: re.sub('[,\.!?-]', '', x))
baseCompleta['Comentario'] = baseCompleta['Comentario'].map(lambda x: normalize('NFKD', x.lower()).encode('ASCII','ignore').decode('ASCII'))

resultado = []

for i in baseCompleta.index:
    fraseComStemming = aplicandoStemmerFrase(baseCompleta["Comentario"][i])
    novaFrase = extratorPalavras(fraseComStemming);
    previsao = classificador.classify(novaFrase)
    resultado.append(previsao)

In [233]:
base_oficial = pd.read_csv('sample_data/DesafioAnderson2.csv', sep=';', engine='python', encoding='utf8')
base_oficial.insert(loc=0, column='Classificação', value=resultado)
base_oficial.to_csv('resultado_final.csv', sep=';', encoding='utf8')